<h1 style="background-color: rgb(236,138,1); height:40px; width: auto; border-radius: 5px; text-align: center">Introduction</h1>

<p style="font-family: 'Menlo','Consolas'; font-size: 13px; text-align: center">
This project will focus on historical sales data from 45 Walmart stores located in different regions.<br>
Each store is defined by a specific type, size and contains many departments.<br>
My goal is to use available data for be able to predict the sales for each department in each store.<br>
Special point is the presence of holiday markdown events included in the dataset.<br>
These markdowns are known to affect sales, it will make prediction which how departments are affected harder.
</p>

<h1 style="background-color: rgb(236,138,1); height:40px; width: auto; border-radius: 5px; text-align: center">Libraries and Tools for the project</h1>

<h2>Data Handling</h2>
<ul>
    <li style="font-family: 'Monaco'; font-size: 15px; text-align: center; text-decoration: underline; list-style-type: none;"><b>Pandas</b></li>
    <p style="font-family: 'Menlo','Consolas'; font-size: 13px; text-align: center">
    Pandas is a powerful data manipulation and analysis library.<br>
    It provides two primary data structures:<br>
    DataFrame: A 2D labeled data structure with columns of potentially different types.<br>
    Series: A 1D array-like object.
    </p>
    <li style="font-family: 'Monaco'; font-size: 15px; text-align: center; text-decoration: underline; list-style-type: none;"><b>NumPy</b></li>
    <p style="font-family: 'Menlo','Consolas'; font-size: 13px; text-align: center">
    NumPy is a library used for numerical computing.<br>
    It provides support for large, multi-dimensional arrays and matrices, along with a collection of mathematical functions to operate on.<br>
    </p>
</ul>

In [1]:
# Data handling
import pandas as pd
import numpy as np

# Viz
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff

# Models
import xgboost as xgb
import catboost as cb
import lightgbm as lgb
from sklearn import model_selection, metrics
from sklearn import linear_model, ensemble
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

# Remove warnings
import warnings
warnings.filterwarnings('ignore') 


/workspaces/Waltrends/.conda/lib/python3.8/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


<hr>
<h2>Data Visualization</h2>
<ul>
    <li style="font-family: 'Monaco'; font-size: 15px; text-align: center; text-decoration: underline; list-style-type: none;"><b>Plotly Express</b></li>
    <p style="font-family: 'Menlo','Consolas'; font-size: 13px; text-align: center">
    Plotly Express is a high-level interface for creating interactive plots easily.<br>
    It simplifies the process of creating visualizations like bar charts, scatter plots, line charts... with minimal code.
    </p>
    <li style="font-family: 'Monaco'; font-size: 15px; text-align: center; text-decoration: underline; list-style-type: none;"><b>Plotly Graph Objects</b></li>
    <p style="font-family: 'Menlo','Consolas'; font-size: 13px; text-align: center">
    Is a lower-level Plotly API for creating more customized and complex visualizations.<br>
    It allows for more detailed control over each component of the graph, such as layout and styling.
    </p>
    <li style="font-family: 'Monaco'; font-size: 15px; text-align: center; text-decoration: underline; list-style-type: none;"><b>Plotly Figure Factory</b></li>
    <p style="font-family: 'Menlo','Consolas'; font-size: 13px; text-align: center">
    Figure factory provides helper functions for creating complex plots like heatmaps, dendrograms, and tables that are otherwise more complex to build manually.
    </p>
</ul>

<hr>
<p style="font-family: 'Menlo','Consolas'; font-size: 13px; text-align: center">
Declaration of several variables with use of pandas for make them dataframe for later use
</p>

In [2]:
features = pd.read_csv('features.csv')
train = pd.read_csv('train.csv')
stores = pd.read_csv('stores.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sampleSubmission.csv')

<hr>
<p style="font-family: 'Menlo','Consolas'; font-size: 13px; text-align: center">
Merge two DataFrames (features and stores) with inner join on the selected field (Store).<br>
Only the matched lines in features and stores are include in the result.<br>
New DataFrame created (feature_store)<br>
</p>

In [3]:
feature_store = features.merge(stores, how='inner', on = "Store")

<p>

</p>

<hr>
<p style="font-family: 'Menlo','Consolas'; font-size: 13px; text-align: center">
Create two Dataframes (train_df and test_df) by merging feature_store on 3 fields (Store, Date and IsHoliday)<br>
In same time i reset the records index preparing for further analysis
</p>

In [4]:
train_df = train.merge(feature_store, how='inner', on = ['Store','Date','IsHoliday']).sort_values(by=['Store','Dept','Date']).reset_index(drop=True)

test_df = test.merge(feature_store, how='inner', on = ['Store','Date','IsHoliday']).sort_values(by = ['Store','Dept','Date']).reset_index(drop=True)

In [5]:
feature_store = features.merge(stores, how='inner', on = "Store")

# Convert date field to datetime format
feature_store['Date'] = pd.to_datetime(feature_store['Date'])
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

# Add datetime features
feature_store['Day'] = feature_store['Date'].dt.day
feature_store['Week'] = feature_store['Date'].dt.isocalendar().week
feature_store['Month'] = feature_store['Date'].dt.month
feature_store['Year'] = feature_store['Date'].dt.year

In [6]:
train_df = train.merge(feature_store, how='inner', on = ['Store','Date','IsHoliday']).sort_values(by=['Store','Dept','Date']).reset_index(drop=True)

In [7]:
test_df = test.merge(feature_store, how='inner', on = ['Store','Date','IsHoliday']).sort_values(by = ['Store','Dept','Date']).reset_index(drop=True)

In [8]:
train_df.describe().T.style.bar(subset=['mean'], color='#205ff2')\
                            .set_caption("Stats Summary of Numeric Variables")\
                            .background_gradient(subset=['min'], cmap='Reds')\
                            .background_gradient(subset=['max'], cmap='Greens')\
                            .background_gradient(subset=['std'], cmap='GnBu')\
                            .background_gradient(subset=['50%'], cmap='GnBu')

,count,mean,std,min,25%,50%,75%,max
Store,421570.000000,22.200546,12.785297,1.000000,11.000000,22.000000,33.000000,45.000000
Dept,421570.000000,44.260317,30.492054,1.000000,18.000000,37.000000,74.000000,99.000000
Weekly_Sales,421570.000000,15981.258123,22711.183519,-4988.940000,2079.650000,7612.030000,20205.852500,693099.360000
Temperature,421570.000000,60.090059,18.447931,-2.060000,46.680000,62.090000,74.280000,100.140000
Fuel_Price,421570.000000,3.361027,0.458515,2.472000,2.933000,3.452000,3.738000,4.468000
MarkDown1,150681.000000,7246.420196,8291.221345,0.270000,2240.270000,5347.450000,9210.900000,88646.760000
MarkDown2,111248.000000,3334.628621,9475.357325,-265.760000,41.600000,192.000000,1926.940000,104519.540000
MarkDown3,137091.000000,1439.421384,9623.078290,-29.100000,5.080000,24.600000,103.990000,141630.610000
MarkDown4,134967.000000,3383.168256,6292.384031,0.220000,504.220000,1481.310000,3595.040000,67474.850000
MarkDown5,151432.000000,4628.975079,5962.887455,135.160000,1878.440000,3359.450000,5563.800000,108519.280000


<hr>

In [9]:
palletes = {
   'continuos':{'blues': ['#03045E', '#023E8A', '#0077B6', '#0077B6', '#0096C7', '#00B4D8', '#48CAE4', '#90E0EF', '#ADE8F4', '#CAF0F8'],
                'green_n_blues': ['#D9ED92', '#B5E48C', '#99D98C', '#76C893', '#52B69A', '#34A0A4', '#168AAD', '#1A759F', '#1E6091', '#184E77']
               }
            }

In [10]:
template = dict(layout=go.Layout(font=dict(family="Enriqueta", size=12))) # Cabin | Franklin Bold

In [15]:
df_weeks = train_df.groupby('Week').sum()

fig = px.line(data_frame=df_weeks, x=df_weeks.index, y='Weekly_Sales', 
              template='simple_white', 
              labels={'Weekly_Sales' : 'Total Sales', 'x' : 'Weeks'})

fig.update_layout(
    template=template, 
    title={'text':'<b>Volume of sales over the year</b>', 'x': 0.075},
    xaxis=dict(tickmode='linear', showline=True), 
    yaxis=dict(showline=True))

fig.add_annotation(
    x=0, y=-0.2, 
    align='left', 
    font=dict(size=12),
    textangle=0, 
    xref="paper", 
    yref="paper", 
    showarrow=False,
    text="<span style='font-size:16px;'><b><i>Result</b></i>: <i>Sales seems to be stable throughout the year, fall from week 42 followed by resurgence at end.</i>", )